In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import os.path, sys

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageEnhance

print(tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Creating the datasets


In [ ]:
train_ds = image_dataset_from_directory(
    directory = '/content/drive/MyDrive/Train',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=16,
    image_size=(256, 256)
)

validation_ds = image_dataset_from_directory(
    directory = '/content/drive/MyDrive/Validate',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=16,
    image_size=(256, 256)
)

test_ds = image_dataset_from_directory(
    directory = '/content/drive/MyDrive/Test',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=16,
    image_size=(256, 256)
)


# Display random selection of traning images with labels

In [ ]:
plt.figure(figsize=(18, 18))
for images, labels in train_ds.take(1):
  for i in range(16):
    ax = plt.subplot(4, 4, i+1)
    plt.imshow(tf.reshape(images[i], [256, 256]))
    plt.title(train_ds.class_names[tf.argmax(labels[i])])
    plt.axis("off")

# Create sharpening layering

In [ ]:
class Sharpen(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(Sharpen, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        self.kernel = np.array([[-2, -2, -2],
                                [-2, 17, -2],
                                [-2, -2, -2]])
        self.kernel = tf.expand_dims(self.kernel, -1)
        self.kernel = tf.expand_dims(self.kernel, -1)
        self.kernel = tf.cast(self.kernel, tf.float32)

    def call(self, input_):
        return tf.nn.conv2d(input_, self.kernel, strides=1, padding='SAME')

# Define the sequential model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(256, 256, 1)),
        layers.Rescaling(1./255),
        Sharpen(num_outputs = (256, 256, 1)),
        layers.Conv2D(16, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(62, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(256, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(512, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(4, activation='softmax'),

    ]
)
print(model.summary())

# Define inception model

In [ ]:
def inception_module(layer_in, c1, c3_in, c3_out, c5_in, c5_out, mp_out):
  conv1 = layers.Conv2D(c1, 1, padding='same', activation='relu')(layer_in)
  conv1 = layers.BatchNormalization()(conv1)

  conv3 = layers.Conv2D(c3_in, 1, padding='same', activation='relu')(layer_in)
  conv3 = layers.BatchNormalization()(conv3)
  conv3 = layers.Conv2D(c3_out, 3, padding='same', activation='relu')(conv3)
  conv3 = layers.BatchNormalization()(conv3)

  conv5 = layers.Conv2D(c5_in, 1, padding='same', activation='relu')(layer_in)
  conv5 = layers.BatchNormalization()(conv5)
  conv5 = layers.Conv2D(c5_out, 5, padding='same', activation='relu')(conv5)
  conv5 = layers.BatchNormalization()(conv5)

  pool = layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
  pool = layers.Conv2D(mp_out, 1, padding='same', activation='relu')(pool)
  pool = layers.BatchNormalization()(pool)

  layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)

  return layer_out

In [ ]:
#start_channels
N = 2

# dropout
dr = 0.4

# l2 regularisation parameter
l2 = 0.001

# number of channels
c1, c3_in, c3_out, c5_in, c5_out, mp_out = 4*N, N, 2*N, 6*N, 8*N, 2*N

inputs = keras.Input(shape=(256, 256, 1))
x = Sharpen(num_outputs=(256, 256, 1))(inputs)
x = layers.Conv2D(2*N, 7, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size = (3,3), padding='same')(x)
x = layers.Conv2D(N, 1, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(N, 3, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size = (3,3), padding='same')(x)
x = inception_module(x, c1, c3_in, c3_out, c5_in, c5_out, mp_out)
x = inception_module(x, c1, c3_in, c3_out, c5_in, c5_out, mp_out)
x = inception_module(x, c1, c3_in, c3_out, c5_in, c5_out, mp_out)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(8*N, activation='relu', kernel_regularizer=regularizers.l2(l2))(x)
x = layers.Dropout(dr)(x)
x = layers.Dense(4*N, activation='relu', kernel_regularizer=regularizers.l2(l2))(x)
x = layers.Dropout(dr)(x)
outputs = layers.Dense(4, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

# Compile and train model

In [ ]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(lr=1e-4),
    metrics=["accuracy"],
)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
history = model.fit(train_ds, epochs=100, verbose=1, callbacks=[model_checkpoint_callback], validation_data=validation_ds)

model.load_weights(checkpoint_filepath)

# Plot accuracy over each epoch


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validate'])
plt.grid()
plt.show()

# Plot loss over each epoch

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validate'])
plt.grid()
plt.show()

# Evaluate model on test and validation datasets

In [ ]:
model.evaluate(test_ds, verbose=1)
model.evaluate(validation_ds, verbose=1)


# Plot confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

predictions_initial = model.predict(test_ds)
predictions = np.array([])
labels =  np.array([])

test_ds.reset()
wrong_predictions_array = np.array([])
for i in range(test_ds.__len__()):
  image = test_ds
  labels = np.concatenate([labels, np.argmax(test_ds.next()[1], axis=-1)])
  predictions = np.concatenate([predictions, [np.argmax(predictions_initial[i], axis=0)]])

cm = confusion_matrix(labels, predictions)

fig, ax  = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(cm, cmap='Blues')
tick_labels = [''] + classes
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(x=j, y=i,s=cm[i, j], va='center', ha='center', size='xx-large')

ax.set_xticklabels(tick_labels)
ax.set_yticklabels(tick_labels)
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.show()